# 不依赖langchain的实现

## 需要安装的包

In [2]:
%%time
%%capture

!pip install faiss-gpu
!pip install sentence_transformers
!pip install openai
!pip install python-dotenv
!pip install pandas

CPU times: user 36.5 ms, sys: 5.41 ms, total: 41.9 ms
Wall time: 12.3 s


In [3]:
%%time

import os
from openai import OpenAI
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

CPU times: user 1.64 s, sys: 290 ms, total: 1.93 s
Wall time: 1.61 s


## 加载环境变量

In [4]:
%load_ext dotenv
%dotenv

token = os.environ.get('ONE_API_TOKEN')
base_url=os.environ.get('ONE_API_URL')

base_url

'http://192.168.0.72:3000/v1'

## 加载嵌入模型

In [5]:
%%time

encoder = SentenceTransformer("/models/bge-m3")

CPU times: user 2.01 s, sys: 1.21 s, total: 3.21 s
Wall time: 7.21 s


## 编造数据

In [6]:
data = [['《西游记》是中国古典小说四大名著之一，又称《大唐西游记》', '小说'],
['牛顿的主要著作是《数学原理》。这部著作于1687年首次出版，被认为是自然科学史上最重要的著作之一。', '数学'],
['东京迪斯尼乐园（Tokyo Disneyland）和东京迪士尼海洋（Tokyo DisneySea）是东京都内的两座迪士尼度假区。', '旅游'],
['电视连续剧《西游记》（1986年版）的主题曲是《云宫迅音》。', '音乐'],
['《西游记》主要内容围绕着僧人唐僧师徒四人（唐僧、孙悟空、猪八戒和沙悟净）西天取经的故事展开。他们历经九九八十一难，克服了各种艰险和困难，最终到达印度取回佛经。', '小说'],
['《西游记》电视连续剧讲述了唐僧师徒四人历经千辛万苦，斗妖除魔，取经西天的壮举，以及他们之间深厚的师徒情谊和对信仰与人性的思考。','电视剧']]
df = pd.DataFrame(data, columns = ['文本', '分类'])

df

,文本,分类
0,《西游记》是中国古典小说四大名著之一，又称《大唐西游记》,小说
1,牛顿的主要著作是《数学原理》。这部著作于1687年首次出版，被认为是自然科学史上最重要的著作之一。,数学
2,东京迪斯尼乐园（Tokyo Disneyland）和东京迪士尼海洋（Tokyo Disney...,旅游
3,电视连续剧《西游记》（1986年版）的主题曲是《云宫迅音》。,音乐
4,《西游记》主要内容围绕着僧人唐僧师徒四人（唐僧、孙悟空、猪八戒和沙悟净）西天取经的故事展开。...,小说
5,《西游记》电视连续剧讲述了唐僧师徒四人历经千辛万苦，斗妖除魔，取经西天的壮举，以及他们之间深...,电视剧


## 创建向量

简化处理，取消文档分片过程。

In [7]:
%%time

text = df['文本']
vectors = encoder.encode(text)

CPU times: user 537 ms, sys: 48 ms, total: 585 ms
Wall time: 588 ms


## 创建faiss索引

In [8]:
%%time

vector_dimension = vectors.shape[1] # 1024
index = faiss.IndexFlatL2(vector_dimension)
faiss.normalize_L2(vectors)
index.add(vectors)

CPU times: user 2.66 ms, sys: 0 ns, total: 2.66 ms
Wall time: 1.32 ms


## 创建查询向量

In [9]:
%%time

search_text = '西游记演了啥?'
search_vector = encoder.encode(search_text)
_vector = np.array([search_vector])
faiss.normalize_L2(_vector)

CPU times: user 20.2 ms, sys: 4.12 ms, total: 24.3 ms
Wall time: 26.4 ms


## 搜索最近相邻

In [20]:
%%time

distances, ann = index.search(_vector, k=3)

results = pd.DataFrame({'distances': distances[0], 'ann': ann[0]})

merge=pd.merge(results,df,left_on='ann',right_index=True)
merge

CPU times: user 2.38 ms, sys: 0 ns, total: 2.38 ms
Wall time: 2.52 ms


,distances,ann,文本,分类
0,0.571907,4,《西游记》主要内容围绕着僧人唐僧师徒四人（唐僧、孙悟空、猪八戒和沙悟净）西天取经的故事展开。...,小说
1,0.603106,5,《西游记》电视连续剧讲述了唐僧师徒四人历经千辛万苦，斗妖除魔，取经西天的壮举，以及他们之间深...,电视剧
2,0.698558,0,《西游记》是中国古典小说四大名著之一，又称《大唐西游记》,小说


In [10]:
merge['文本'].values

array(['《西游记》主要内容围绕着僧人唐僧师徒四人（唐僧、孙悟空、猪八戒和沙悟净）西天取经的故事展开。他们历经九九八十一难，克服了各种艰险和困难，最终到达印度取回佛经。',
       '《西游记》电视连续剧讲述了唐僧师徒四人历经千辛万苦，斗妖除魔，取经西天的壮举，以及他们之间深厚的师徒情谊和对信仰与人性的思考。',
       '《西游记》是中国古典小说四大名著之一，又称《大唐西游记》'], dtype=object)

## reranker

In [11]:
merged_items = [[search_text, x] for x in merge['文本'].values]
merged_items

[['西游记演了啥?',
  '《西游记》主要内容围绕着僧人唐僧师徒四人（唐僧、孙悟空、猪八戒和沙悟净）西天取经的故事展开。他们历经九九八十一难，克服了各种艰险和困难，最终到达印度取回佛经。'],
 ['西游记演了啥?',
  '《西游记》电视连续剧讲述了唐僧师徒四人历经千辛万苦，斗妖除魔，取经西天的壮举，以及他们之间深厚的师徒情谊和对信仰与人性的思考。'],
 ['西游记演了啥?', '《西游记》是中国古典小说四大名著之一，又称《大唐西游记》']]

In [13]:
%%time
%%capture

import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('/models/bge-reranker-v2-m3')
model = AutoModelForSequenceClassification.from_pretrained('/models/bge-reranker-v2-m3')
model.eval()

CPU times: user 1.18 s, sys: 1.03 s, total: 2.21 s
Wall time: 10.9 s


XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(8194, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, out_f

In [19]:
%%time

with torch.no_grad():
    inputs = tokenizer(merged_items, padding=True, truncation=True, return_tensors='pt', max_length=512)
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    print(scores)

tensor([ 0.3166, -0.2007, -2.1656])
CPU times: user 1.97 s, sys: 0 ns, total: 1.97 s
Wall time: 508 ms


## LLM 生成回答

TODO: 没有直接使用 reranker的数据，而是手动取前两条数据

In [21]:
client = OpenAI(api_key=token, base_url=base_url)

def process_stream(stream):
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            print(chunk.choices[0].delta.content, end="")

stream = client.chat.completions.create(
    model="xiaoyu",
    messages=[{"role": "user", "content": "你谁啊？"}],
    stream=True,
)

process_stream(stream)

我是小羽，来自羽胜科技。我是一个大型语言模型，被设计用来回答各种问题、提供信息和进行对话。有什么可以帮助你的吗？

In [22]:
str='这是有关{topic}的回答上下文。'
topic='游戏'
str.format(topic=topic)

'这是有关游戏的回答上下文。'

In [26]:
prompt="""
Answer the following question based only on the provided context:
<context>
{context}
</context>

Question: {input}
"""

input='西游记演了啥?'
context=[
    '西游记》主要内容围绕着僧人唐僧师徒四人（唐僧、孙悟空、猪八戒和沙悟净）西天取经的故事展开。他们历经九九八十一难，克服了各种艰险和困难，最终到达印度取回佛经。',
    '《西游记》电视连续剧讲述了唐僧师徒四人历经千辛万苦，斗妖除魔，取经西天的壮举，以及他们之间深厚的师徒情谊和对信仰与人性的思考。'
]
context = '\n'.join(context)

stream = client.chat.completions.create(
    model="xiaoyu",
    messages=[{"role": "user", "content": prompt.format(input=input,context=context)}],
    stream=True,
)

process_stream(stream)

《西游记》电视连续剧主要讲述了唐僧师徒四人（唐僧、孙悟空、猪八戒和沙悟净）取经的故事，他们在取经过程中面对各种妖怪和困难，展现了他们的勇气、智慧和团队合作精神。电视剧着重描绘了他们之间的师徒情谊以及对信仰和人性的深刻探讨。